In [138]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [139]:
#Sex male:0, female:1
#Embarked S:0, C:1, Q:2

train = pd.read_csv('train.csv').replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test = pd.read_csv('test.csv').replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

y_test = pd.read_csv('gender_submission.csv')

In [140]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.6+ KB


In [141]:

train["Age"].fillna(train.Age.mean(),inplace=True)
train["Embarked"].fillna(train.Embarked.mean(),inplace=True)

test["Age"].fillna(train.Age.mean(),inplace=True)
test["Embarked"].fillna(train.Embarked.mean(),inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

#列削除
train.drop("Name",axis=1,inplace=True)
train.drop("Cabin",axis=1,inplace=True)
train.drop("Ticket",axis=1,inplace=True)

test.drop("Name",axis=1,inplace=True)
test.drop("Cabin",axis=1,inplace=True)
test.drop("Ticket",axis=1,inplace=True)


train_data = train.values
x_train = train_data[:, 2:] # Pclass以降の変数
y_train  = train_data[:, 1]  # 正解データ

test_data = test.values
x_test = test_data[:, 1:] # Pclass以降の変数

y_test = y_test.values[:,1]

In [142]:
#モデル作成
from sklearn import tree
from sklearn.grid_search import GridSearchCV

tuned_parameters = {'max_depth':  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], # 木の深さを1-10で
                    'max_leaf_nodes':  [2,4,6,8,10] # 最大終端ノード数を2,4,6,8,10で
                   }
 
# 上記で用意したパラメーターごとに交差検証を実施。最適な木の深さを確認する。
clf = GridSearchCV(tree.DecisionTreeClassifier(random_state=5,splitter='best'), tuned_parameters, scoring="accuracy",cv=5, n_jobs=1)
clf = clf.fit(x_train, y_train) # モデル作成!

print("complete!")

complete!


In [143]:
print("Best Parameter: {}".format(clf.best_params_))
print("Best Parameterでの検証用データの精度: {:.2f}".format(clf.score(x_test, y_test)))
print("Best Parameterで交差検証した精度の平均（訓練データ）: {:.2f}".format(clf.best_score_))

Best Parameter: {'max_depth': 3, 'max_leaf_nodes': 8}
Best Parameterでの検証用データの精度: 0.96
Best Parameterで交差検証した精度の平均（訓練データ）: 0.82


In [144]:
clf = tree.DecisionTreeClassifier(max_depth=3,max_leaf_nodes=8)
clf = clf.fit(x_train,y_train)



print(clf.feature_importances_)

[0.21121438 0.62207309 0.07202059 0.04444026 0.         0.05025168
 0.        ]


In [145]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
y_pred = clf.predict(x_test)
print(confusion_matrix(y_test,y_pred))

0.958904109589041
0.9210526315789473
[[260   6]
 [ 12 140]]


In [146]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null int64
dtypes: float64(2), int64(6)
memory usage: 26.2 KB


In [113]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
PassengerId    418 non-null int64
Survived       418 non-null int64
dtypes: int64(2)
memory usage: 6.6 KB
